# Lead Time Prediction Model
This notebook aims to predict the delivery lead time for orders based on various features related to the supply chain process. The goal is to help minimize delivery delays by accurately estimating lead times.


## Data Loading and Preparation
In this section, we load the dataset and prepare it for analysis, including handling missing values and converting data types as necessary.


In [1]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('USAID_Health_Comodity_Delivery_CleanUp.csv')
# Convert the columns to datetime format if they are not already
df['Latest Actual Delivery Date'] = pd.to_datetime(df['Latest Actual Delivery Date'])
df['Order Entry Date'] = pd.to_datetime(df['Order Entry Date'])

# Create the new column
df['Actual Lead Time'] = df['Latest Actual Delivery Date'] - df['Order Entry Date']
df['Actual Lead Time'] = df['Actual Lead Time'].dt.days


/tmp/ipykernel_25717/3540978686.py:4: DtypeWarning: Columns (50,51,52,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('USAID_Health_Comodity_Delivery_CleanUp.csv')


## Feature and Target Variable Definition
Modify the `features` and `target` variables as needed to fit your analysis. The current setup is aimed at predicting the Actual Lead Time based on selected features from the dataset.


In [2]:

# Define the features and target for the model
features = ['Country', 'Transportation Mode', 'Order Type', 'Fulfillment Method', 'Product Category', 'Vendor Incoterm', 'Reason Code', 'Item Tracer Category','Quality Assurance']
target = ['Actual Lead Time']


An overview of what the dataframe would look like

In [3]:
df = df[features + target]
df['Reason Code'] = df['Reason Code'].str.slice(0, 2)
df=df.dropna()

In [4]:
df

,Country,Transportation Mode,Order Type,Fulfillment Method,Product Category,Vendor Incoterm,Reason Code,Item Tracer Category,Quality Assurance,Actual Lead Time
34575,Uganda,Land,Purchase Order,Direct Drop,Malaria Pharmaceuticals,DDP,SD,ACTs,119.0,325
34576,Zambia,Land,Purchase Order,Direct Drop,Malaria Pharmaceuticals,DDP-ZAMBIA,SD,ACTs,132.0,260
34588,Kenya,Air,Purchase Order,Direct Drop,Malaria Pharmaceuticals,FCA-CHINA,AD,Severe Malaria Meds,54.0,503
34594,Kenya,Air,Purchase Order,Direct Drop,Malaria Pharmaceuticals,FCA-CHINA,AD,Severe Malaria Meds,60.0,504
34598,Tanzania,Land,Purchase Order,Direct Drop,Long Lasting Insecticide Treated Nets (LLINs),DAP,SD,LLINs,36.0,516
...,...,...,...,...,...,...,...,...,...,...
38497,Zambia,Sea,Purchase Order,Direct Drop,Malaria Pharmaceuticals,FCA,AD,Severe Malaria Meds,95.0,280
38498,Zambia,Air,Purchase Order,Direct Drop,Malaria Pharmaceuticals,FCA,SD,Severe Malaria Meds,17.0,303
38499,Niger,Air,Purchase Order,Direct Drop,Malaria Pharmaceuticals,FCA,AD,Severe Malaria Meds,42.0,206
38501,Senegal,Sea,Purchase Order,Direct Drop,Long Lasting Insecticide Treated Nets (LLINs),FCA,AD,LLINs,45.0,694


## Data Preprocessing
Before training our models, we need to preprocess the data. This includes encoding categorical variables, handling missing values, in this cased, if a column has an empty cell, drop it.


In [5]:
# Convert columns to 'category' type
for col in features:
    if col == 'Quality Assurance':
        continue
    df[col] = df[col].astype('category')

Since we changing everything to label, create a dictionary to keep track of the numeric label

In [6]:
from sklearn.preprocessing import LabelEncoder

# Initialize a LabelEncoder
le = LabelEncoder()

# Dictionary to hold the mappings for each column
mapping = {}

# Apply the LabelEncoder to the categorical columns
for col in features:
    if col == 'Quality Assurance':
        continue

    df[col] = le.fit_transform(df[col])
    mapping[col] = dict(zip(le.transform(le.classes_), le.classes_))  # Reverse the mappings

# Print the mappings
for col, map_dict in mapping.items():
    print(f"{col}: {map_dict}")

Country: {0: 'Angola', 1: 'Belgium', 2: 'Benin', 3: 'Burkina Faso', 4: 'Burundi', 5: 'Cambodia', 6: 'Cameroon', 7: 'Congo DRC', 8: "Côte d'Ivoire", 9: 'Ethiopia', 10: 'Ghana', 11: 'Guinea', 12: 'Kenya', 13: 'Laos', 14: 'Liberia', 15: 'Madagascar', 16: 'Malawi', 17: 'Mali', 18: 'Mozambique', 19: 'Myanmar', 20: 'Niger', 21: 'Nigeria', 22: 'Rwanda', 23: 'Senegal', 24: 'Sierra Leone', 25: 'South Africa', 26: 'South Sudan', 27: 'Tanzania', 28: 'Thailand', 29: 'Uganda', 30: 'Zambia', 31: 'Zimbabwe'}
Transportation Mode: {0: 'Air', 1: 'Land', 2: 'Multiple', 3: 'Sea'}
Order Type: {0: 'Purchase Order', 1: 'Replenishment Order'}
Fulfillment Method: {0: 'Direct Drop'}
Product Category: {0: 'Essential Medicines', 1: 'Laboratory Consumables', 2: 'Laboratory Equipment', 3: 'Long Lasting Insecticide Treated Nets (LLINs)', 4: 'Malaria Pharmaceuticals', 5: 'Malaria Rapid Diagnostic Test (RDTs)'}
Vendor Incoterm: {0: 'DAP', 1: 'DDP', 2: 'DDP (ZAMBIA)', 3: 'DDP-NIGERIA', 4: 'DDP-ZAMBIA', 5: 'DDP/UGANDA',

An overview of what the dataframe would look like after processing and before being dump to machine learning model

In [7]:
df

,Country,Transportation Mode,Order Type,Fulfillment Method,Product Category,Vendor Incoterm,Reason Code,Item Tracer Category,Quality Assurance,Actual Lead Time
34575,29,1,0,0,4,1,1,0,119.0,325
34576,30,1,0,0,4,4,1,0,132.0,260
34588,12,0,0,0,4,9,0,7,54.0,503
34594,12,0,0,0,4,9,0,7,60.0,504
34598,27,1,0,0,3,0,1,1,36.0,516
...,...,...,...,...,...,...,...,...,...,...
38497,30,3,0,0,4,7,0,7,95.0,280
38498,30,0,0,0,4,7,1,7,17.0,303
38499,20,0,0,0,4,7,0,7,42.0,206
38501,23,3,0,0,3,7,0,1,45.0,694


Example of listing attribute that have the highest averge lead time

In [8]:
# List of categorical columns
categorical_columns = [feature for feature in features if feature != 'Quality Assurance']

# Empty DataFrame to store the results
results = pd.DataFrame(columns=['Feature', 'Category', 'Average Lead Time'])

# Iterate over the categorical columns
for col in categorical_columns:
    # Calculate the average 'Actual Lead Time' for each category within the current feature
    average_lead_time = df.groupby(col)['Actual Lead Time'].mean()
    
    # Create a temporary DataFrame with the results
    temp_df = pd.DataFrame({
        'Feature': col,
        'Category': average_lead_time.index,
        'Average Lead Time': average_lead_time.values
    })
    
    # Replace the labels with their original categories using the mapping dictionary
    temp_df['Category'] = temp_df['Category'].map(mapping[col])
    
    # Append the temporary DataFrame to the results DataFrame
    results = pd.concat([results, temp_df], ignore_index=True)

# Sort the results DataFrame by 'Average Lead Time' in descending order and get the top 10
top_10 = results.sort_values(by='Average Lead Time', ascending=False).head(10)

# Print the top 10
print(top_10)

                Feature                Category  Average Lead Time
48      Vendor Incoterm             DDP-NIGERIA         517.500000
19              Country                 Myanmar         491.727273
59      Vendor Incoterm                     FCR         477.666667
45      Vendor Incoterm                     DAP         448.280000
7               Country               Congo DRC         433.808889
27              Country                Tanzania         432.591398
40     Product Category  Laboratory Consumables         420.000000
12              Country                   Kenya         416.586207
46      Vendor Incoterm                     DDP         415.500000
34  Transportation Mode                Multiple         404.375000


/tmp/ipykernel_25717/1864760275.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, temp_df], ignore_index=True)


## Model Training and Evaluation
In this section, we define several machine learning models to predict the Actual Lead Time. We then train these models on our dataset and evaluate their performance.


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from math import sqrt
import pandas as pd

# Define the features and the target
X = df[features]
y = df['Actual Lead Time']

# Define the categorical features (all features except 'Quality Assurance')
categorical_features = [feature for feature in features if feature != 'Quality Assurance']

# Create a transformer for the categorical features
categorical_transformer = OneHotEncoder(drop='first')

# Create a preprocessor
preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_features)
])

# Transform the features
X = preprocessor.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define the models
models = [
    ('Linear Regression', LinearRegression()),
    ('Decision Tree', DecisionTreeRegressor(random_state=0)),
    ('Random Forest', RandomForestRegressor(random_state=0)),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=0)),
    ('Support Vector', SVR())
]

# Loop through the models
for name, model in models:
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate the performance metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Print the performance metrics
    print(f'{name}:')
    print(f'Root Mean Squared Error: {rmse}')
    print(f'R-squared: {r2}\n')
    '''
    # Print feature importance for models that have the feature_importances_ attribute
    if hasattr(model, 'feature_importances_'):
        print(f'Feature Importance for {name}:')
        # Get feature names after one-hot encoding
        feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
        feature_importances = pd.Series(model.feature_importances_, index=feature_names)
        print(feature_importances.nlargest(10))  # Print only the top 10 features
        print('\n')
    '''

Linear Regression:
Root Mean Squared Error: 99.76701646344684
R-squared: 0.35357230738923184

Decision Tree:
Root Mean Squared Error: 101.71994301214409
R-squared: 0.3280171337562934

Random Forest:
Root Mean Squared Error: 99.23720361253888
R-squared: 0.36041978715751377

Gradient Boosting:
Root Mean Squared Error: 99.308575931229
R-squared: 0.35949947226621404

Support Vector:
Root Mean Squared Error: 113.91986015975627
R-squared: 0.15716052127027758

